In [2]:
import h3
import os
import polars as pl
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import folium

In [3]:
# create data directories
os.makedirs("./data/objects", exist_ok=True)
os.makedirs("./data/people", exist_ok=True)

In [4]:
# delete all files in data directories
for file in os.listdir("./data/objects"):
    os.remove(os.path.join("./data/objects", file))
for file in os.listdir("./data/people"):
    os.remove(os.path.join("./data/people", file))


In [8]:
objects = pl.read_parquet("./data/objects/")
people = pl.read_parquet("./data/people/")
orders = pl.read_parquet("./data/sites/10596657-995b-5410-8033-2f6d7dd6c71f/orders")
lines = pl.read_parquet("./data/sites/10596657-995b-5410-8033-2f6d7dd6c71f/lines")

orders_counts = pl.sql("SELECT status, count(*) as count FROM orders GROUP BY status").collect()
lines_counts = pl.sql("SELECT status, count(*) as count FROM lines GROUP BY status").collect()
people_counts = pl.sql("SELECT role, count(*) as count FROM people GROUP BY role").collect()
object_counts = pl.sql("SELECT label, count(*) as count FROM objects GROUP BY label").collect()

In [ ]:
# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

# Add object counts pie chart
fig.add_trace(
    go.Pie(
        labels=object_counts['label'].to_list(),
        values=object_counts['count'].to_list(),
        name="Objects",
        title="Object Types",
        showlegend=True,
        legendgroup="objects",
    ),
    row=1, col=1
)

# Add people counts pie chart
fig.add_trace(
    go.Pie(
        labels=people_counts['role'].to_list(),
        values=people_counts['count'].to_list(),
        name="People",
        title="People Roles",
        showlegend=True,
        legendgroup="people",
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Data Composition Analysis",
    height=500,
    showlegend=True,
    template="plotly_dark",
)

fig

In [ ]:
lng, lat = -0.13381370382489707, 51.518898098201326
resolution = 6

# Create a map centered on the first person's location
m = folium.Map(
    location=[lat, lng],
    zoom_start=13,
    tiles="CartoDB dark_matter",  # Dark theme map
)

# folium.TileLayer("OpenStreetMap").add_to(m)
# folium.TileLayer(show=False).add_to(m)
# folium.LayerControl().add_to(m)

cell = h3.latlng_to_cell(lat, lng, resolution)
polygon = h3.cells_to_geo([cell])
folium.GeoJson(polygon).add_to(m)

for row in people.select(["position", "role"]).rows():
    lat, lon = row[0]
    folium.Marker(
        location=[lon, lat],
        popup="Person",
        icon=folium.Icon(color="blue" if row[1] == "customer" else "red", icon="user", prefix="fa"),
    ).add_to(m)

m